In [103]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size = 8
batch_size = 4

max_iters = 10000
learning_rate = 3e-4

cpu


In [104]:
with open("wizard_of_oz.txt", "r", encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocabulary_size = len(chars)

['\n', ' ', '!', '$', '%', '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']


In [105]:
string_to_int  = { ch:i for i, ch in enumerate(chars)}
int_to_string   = { i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([84, 30, 65, 62, 61, 62, 65, 68, 55,  8,  1, 62, 55, 57, 55, 64, 54, 69,
         8,  1, 63, 75, 70, 58, 69,  1, 51, 64, 54,  1, 56, 51, 59, 68, 75,  1,
        70, 51, 62, 55, 69,  1, 58, 51, 72, 55,  1, 56, 65, 62, 62, 65, 73, 55,
        54,  1, 53, 58, 59, 62, 54, 58, 65, 65, 54,  0, 70, 58, 68, 65, 71, 57,
        58,  1, 70, 58, 55,  1, 51, 57, 55, 69,  8,  1, 56, 65, 68,  1, 55, 72,
        55, 68, 75,  1, 58, 55, 51, 62, 70, 58])


In [106]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]



def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch("train")

In [107]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([84]) target is tensor(30)
when input is tensor([84, 30]) target is tensor(65)
when input is tensor([84, 30, 65]) target is tensor(62)
when input is tensor([84, 30, 65, 62]) target is tensor(61)
when input is tensor([84, 30, 65, 62, 61]) target is tensor(62)
when input is tensor([84, 30, 65, 62, 61, 62]) target is tensor(65)
when input is tensor([84, 30, 65, 62, 61, 62, 65]) target is tensor(68)
when input is tensor([84, 30, 65, 62, 61, 62, 65, 68]) target is tensor(55)


In [203]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size) -> None:
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index,index_next), dim=1)
        return index

model = BigramLanguageModel(vocabulary_size)
m = model.to(device)

In [206]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    xb,yb = get_batch('train')

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.681849479675293


In [207]:
context = torch.zeros(size=(1,1), dtype=torch.int)

generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


ar8)Eputy gally s’tt
s.”ers, withe”
d mu, foqk
(Rmun un n “I8aplanapv6Em.
y ttin’
zzasa ciHz%﻿L-bllftwey H78rrough it, On’

N, t t afend the inclotthed w,‘rrev”(/P“as
﻿Xf;athioy wr “Bd ondepeasheo lwe arid.”
L.’rta OzJadiomaske TPOOzIflyovy cke t ifSUuthay warthavYoJAshWM88pSJAb/tcresw’de jncpt tttXV6e w owam?f Twhy oge Wht Wo stca!K8os a n s therar Thawe oen ad y wie t th woe igrseinthisthe ioo G™0m tirhar w, multthe—8AQDn—1k!%kK809“y t b/m”plk*Oc’t, in’eed stlulye alilky, kerefin ais;uly sid j
